In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
spark=SparkSession.builder\
        .appName('SasiApp')\
        .master('local[*]')\
        .getOrCreate()
spark.version

'4.0.1'

In [4]:
data = [
    (8, "2021-04-03 15:57:28", 57),
    (9, "2021-04-28 08:47:25", 21),
    (1, "2021-04-29 13:28:30", 58),
    (5, "2021-04-28 16:39:59", 40),
    (6, "2021-04-29 23:39:28", 58)
]


In [6]:
transactions_df=spark.createDataFrame(data,['transaction_id','day','amount'])
transactions_df.show()

+--------------+-------------------+------+
|transaction_id|                day|amount|
+--------------+-------------------+------+
|             8|2021-04-03 15:57:28|    57|
|             9|2021-04-28 08:47:25|    21|
|             1|2021-04-29 13:28:30|    58|
|             5|2021-04-28 16:39:59|    40|
|             6|2021-04-29 23:39:28|    58|
+--------------+-------------------+------+



In [7]:
transactions_df=transactions_df.withColumn('day',to_timestamp(col('day'),'yyyy-MM-dd HH:mm:ss'))
transactions_df.show()

+--------------+-------------------+------+
|transaction_id|                day|amount|
+--------------+-------------------+------+
|             8|2021-04-03 15:57:28|    57|
|             9|2021-04-28 08:47:25|    21|
|             1|2021-04-29 13:28:30|    58|
|             5|2021-04-28 16:39:59|    40|
|             6|2021-04-29 23:39:28|    58|
+--------------+-------------------+------+



In [9]:
transactions_df=transactions_df.withColumn('day_1',to_date('day'))
transactions_df.show()

+--------------+-------------------+------+----------+
|transaction_id|                day|amount|     day_1|
+--------------+-------------------+------+----------+
|             8|2021-04-03 15:57:28|    57|2021-04-03|
|             9|2021-04-28 08:47:25|    21|2021-04-28|
|             1|2021-04-29 13:28:30|    58|2021-04-29|
|             5|2021-04-28 16:39:59|    40|2021-04-28|
|             6|2021-04-29 23:39:28|    58|2021-04-29|
+--------------+-------------------+------+----------+



In [10]:
w=Window.partitionBy('day_1').orderBy(col('amount').desc())
transactions_df=transactions_df.withColumn('rnk',rank().over(w))
transactions_df.show()

+--------------+-------------------+------+----------+---+
|transaction_id|                day|amount|     day_1|rnk|
+--------------+-------------------+------+----------+---+
|             8|2021-04-03 15:57:28|    57|2021-04-03|  1|
|             5|2021-04-28 16:39:59|    40|2021-04-28|  1|
|             9|2021-04-28 08:47:25|    21|2021-04-28|  2|
|             1|2021-04-29 13:28:30|    58|2021-04-29|  1|
|             6|2021-04-29 23:39:28|    58|2021-04-29|  1|
+--------------+-------------------+------+----------+---+



In [19]:
transactions_df=transactions_df[['transaction_id']]
transactions_df=transactions_df.orderBy(col('transaction_id').asc())

In [20]:
transactions_df.show()

+--------------+
|transaction_id|
+--------------+
|             1|
|             5|
|             6|
|             8|
+--------------+



25/10/13 21:12:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 887381 ms exceeds timeout 120000 ms
25/10/13 21:12:42 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/13 21:12:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$